In [ ]:
#!pip install flair

In [8]:
# Uncomment if you need to download data
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

--2021-02-09 14:38:57--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125059 (122K) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>] 122.13K  --.-KB/s    in 0.08s   

2021-02-09 14:38:58 (1.47 MB/s) - ‘train.conll’ saved [125059/125059]

--2021-02-09 14:38:58--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14109 (14K) [text/plain]
Saving to: ‘dev.conll’

dev.conll           100%[===============

In [18]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz

--2021-02-19 09:41:33--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘cc.ru.300.bin.gz’

cc.ru.300.bin.gz      6%[>                   ] 268.25M  11.7MB/s    eta 6m 22s ^C


In [6]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import (
    TokenEmbeddings, WordEmbeddings,
    StackedEmbeddings, CharacterEmbeddings,
    FlairEmbeddings, TransformerWordEmbeddings,
    FastTextEmbeddings, BytePairEmbeddings
)
from flair.visual.training_curves import Plotter
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger

In [7]:
import flair
import torch
# uncomment for cpu-only training
#flair.device = torch.device('cpu')

In [8]:
# 1. get the corpus
corpus: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train.conll', dev_file='dev.conll', test_file='test.conll',
)
print(corpus)

2021-02-21 17:41:00,089 Reading data from .
2021-02-21 17:41:00,089 Train: train.conll
2021-02-21 17:41:00,090 Dev: dev.conll
2021-02-21 17:41:00,090 Test: test.conll
Corpus: 2000 train + 211 dev + 521 test sentences


In [9]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

Dictionary with 14 tags: <unk>, O, B-FILM, B-SONG, B-SINGER, I-SONG, I-SINGER, B-COMPOSER, I-COMPOSER, I-FILM, B-BOOK, I-BOOK, <START>, <STOP>


In [16]:
from hyperopt import hp
from flair.hyperparameter.param_selection import SearchSpace, Parameter

# define your search space
search_space = SearchSpace()
search_space.add(Parameter.EMBEDDINGS, hp.choice, options=[
    StackedEmbeddings([
        WordEmbeddings("ru"),
        FlairEmbeddings("bg-forward"),
        FlairEmbeddings("bg-backward")
    ]), 
    StackedEmbeddings([
        WordEmbeddings("ru"),
        FlairEmbeddings('multi-forward'),
        FlairEmbeddings('multi-backward') 
    ]),
    BytePairEmbeddings('multi')
])
search_space.add(Parameter.HIDDEN_SIZE, hp.choice, options=[32, 64, 128, 256])
search_space.add(Parameter.RNN_LAYERS, hp.choice, options=[1, 2])
search_space.add(Parameter.DROPOUT, hp.uniform, low=0.0, high=0.5)
search_space.add(Parameter.LEARNING_RATE, hp.choice, options=[0.05, 0.1, 0.15, 0.2])
search_space.add(Parameter.MINI_BATCH_SIZE, hp.choice, options=[8, 16, 32, 64])
search_space.add(Parameter.)

Setting dim=300 for multilingual BPEmb


In [25]:
from flair.hyperparameter.param_selection import SequenceTaggerParamSelector, OptimizationValue

In [26]:
param_selector = SequenceTaggerParamSelector(
    corpus,
    tag_type, 
    'resources/results',
    max_epochs=20, 
    training_runs=3,
    optimization_value=OptimizationValue.DEV_SCORE
)

In [ ]:
param_selector.optimize(search_space, max_evals=100)

In [13]:
best_model = SequenceTagger(
    hidden_size=256,
    embeddings=StackedEmbeddings([
        WordEmbeddings("ru"),
        FlairEmbeddings("bg-forward"),
        FlairEmbeddings("bg-backward")
    ]),
    dropout=0.26434517750058717,
    rnn_layers=2,
    tag_type=tag_type,
    tag_dictionary=tag_dictionary
)

In [14]:
trainer = ModelTrainer(best_model, corpus)

In [15]:
trainer.train("./resources/best_model", learning_rate=0.2, mini_batch_size=8, max_epochs=200)

2021-02-21 17:41:37,585 ----------------------------------------------------------------------------------------------------
2021-02-21 17:41:37,586 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('ru')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(4336, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=4336, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(4336, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=4336, bias=True)
      )
    )
  )
  (dropout): Dropout(p=0.26434517750058717, inplace=False)
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4396, out_features=439

{'test_score': 0.7323232323232324,
 'dev_score_history': [0.6342494714587739,
  0.6497890295358649,
  0.6708860759493671,
  0.6877637130801688,
  0.711578947368421,
  0.6708860759493671,
  0.70042194092827,
  0.6877637130801688,
  0.7257383966244725,
  0.7088607594936709,
  0.7257383966244725,
  0.7172995780590716,
  0.759493670886076,
  0.7046413502109705,
  0.7284210526315791,
  0.7426160337552744,
  0.7299578059071729,
  0.7452631578947368,
  0.7299578059071729,
  0.7383966244725737,
  0.7326315789473684,
  0.7326315789473684,
  0.7410526315789474,
  0.7368421052631579,
  0.7172995780590716,
  0.7452631578947368,
  0.7410526315789474,
  0.7452631578947368,
  0.7368421052631579,
  0.7410526315789474,
  0.7452631578947368,
  0.7452631578947368,
  0.7410526315789474,
  0.7452631578947368,
  0.7410526315789474,
  0.7410526315789474,
  0.7410526315789474,
  0.7410526315789474,
  0.7410526315789474,
  0.7368421052631579,
  0.7410526315789474,
  0.7368421052631579,
  0.7368421052631579,
  